<a href="https://colab.research.google.com/github/aetev/Hearth-Stone-Python-Simulator/blob/main/card_enc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import json

# Assuming AtomicCards.json is in your Google Drive, adjust the path if necessary
file_path = '/content/drive/MyDrive/AtomicCards.json'


with open(file_path, 'r') as f:
  data = json.load(f)['data'])

In [11]:
def filter_legal_cards(data):
    legal_commander_data = {}
    for card_name, card_data in data.items():
        try:
            if card_data[0]['legalities']['commander'] == 'Legal':
                legal_commander_data[card_name] = card_data[0] # Store only the first element of card_data
        except (KeyError, IndexError):
            # Handle cases where 'legalities' or 'commander' key is missing or card_data is empty
            pass  # Or print a message, log it, etc.
    return legal_commander_data

filtered_data = filter_legal_cards(data)


In [30]:
items = list(filtered_data.items())

first_value = filtered_data[first_key]
second_key, second_value = items[4]
print("Key:", second_value)
print("text:", second_value['text'])
print("keywords:", second_value['keywords'])

Key: {'colorIdentity': ['G'], 'colors': ['G'], 'convertedManaCost': 5.0, 'edhrecRank': 21546, 'edhrecSaltiness': 0.11, 'firstPrinting': 'MKM', 'foreignData': [{'language': 'German', 'name': 'Mörder in unserer Mitte', 'text': 'Wenn Mörder in unserer Mitte ins Spiel kommt, erzeuge einen 1/1 weißen Mensch-Kreaturenspielstein, einen 1/1 blauen Meervolk-Kreaturenspielstein und einen 1/1 roten Goblin-Kreaturenspielstein. Bestimme dann geheim Mensch, Meervolk oder Goblin.\nOpfere Mörder in unserer Mitte, offenbare den bestimmten Kreaturentyp: Falls ein angreifender Kreaturenspielstein deiner Wahl den bestimmten Typ hat, lege drei +1/+1-Marken auf ihn und er erhält Todesberührung bis zum Ende des Zuges.', 'type': 'Verzauberung'}, {'language': 'Spanish', 'name': 'Un asesino entre nosotros', 'text': 'Cuando Un asesino entre nosotros entre al campo de batalla, crea una ficha de criatura Humano blanca 1/1, una ficha de criatura Tritón azul 1/1 y una ficha de criatura Trasgo roja 1/1. Luego, elige 

KeyError: 'keywords'

In [ ]:
# second_key, second_value = items[1]

items = list(filtered_data.items())
from google.colab import drive
import json

drive.mount('/content/drive')

# Assuming AtomicCards.json is in your Google Drive, adjust the path if necessary
file_path = '/content/drive/MyDrive/AtomicCards.json'


with open(file_path, 'r') as f:
  data = json.load(f)['data']

def filter_legal_cards(data):
    legal_commander_data = {}
    for card_name, card_data in data.items():
        try:
            if card_data[0]['legalities']['commander'] == 'Legal':
                legal_commander_data[card_name] = card_data[0] # Store only the first element of card_data
        except (KeyError, IndexError):
            # Handle cases where 'legalities' or 'commander' key is missing or card_data is empty
            pass  # Or print a message, log it, etc.
    return legal_commander_data

filtered_data = filter_legal_cards(data)

# Get the second item (key-value pair)

if len(items) > 1:

    print("Second Key:", second_key)
    print("Second text:", second_value['text'])
    print("Second keywords:", second_value['keywords'])
else:
    print("The dictionary does not have a second item.")
